In [2]:
from monty.serialization import loadfn, dumpfn
from pymatgen.core import Structure
from pymatgen.io.vasp.sets import MPRelaxSet, MPMetalRelaxSet
from pymatgen.io.vasp import Poscar
from doped.chemical_potentials import CompetingPhases, CompetingPhasesAnalyzer
from pymatgen.entries.computed_entries import ComputedEntry
import os

In [3]:
supercell = Poscar.from_file("doped-defs-AlClO/AlOCl_bulk/vasp_std/CONTCAR")
bulk_supercell = supercell.structure

In [7]:
cp = CompetingPhases(composition='AlClO', extrinsic='Cr', energy_above_hull=0.05)

In [55]:
for entry in cp.entries:
    dir_path = os.path.join(
        "AlClO",
        "AlClO_competing_phases",
        f"{entry.data['formula_pretty']}_EaH_{entry.data['energy_above_hull']}",
        "vasp_std"
    )
    os.makedirs(dir_path, exist_ok=True)

In [56]:
calculated_entries = {}
for entry in cp.entries:
    key =  entry.data['material_id'] + '-' + entry.structure.composition.reduced_formula
    if key in calculated_entries:
        continue
    # if entry.data['summary']['band_gap'] == 0:
    else:
        entries.append(key)
        if entry.data['summary']['band_gap'] == 0:
            mp_relax_set = MPMetalRelaxSet(entry.structure)
            mp_relax_set.write_input(f"AlClO/AlClO_competing_phases/{entry.data['formula_pretty']}_EaH_{entry.data['energy_above_hull']}/vasp_std", poscar=True)
        else:
            mp_relax_set = MPRelaxSet(entry.structure)
            mp_relax_set.write_input(f"AlClO/AlClO_competing_phases/{entry.data['formula_pretty']}_EaH_{entry.data['energy_above_hull']}/vasp_std", poscar=True)
        calculated_entries[key] = entry

In [5]:
cpa = CompetingPhasesAnalyzer(composition='AlClO', entries="AlClO/AlClO_competing_phases")

Parsing vaspruns...: 100%|██████████████████████████████████████████████████████████████| 22/22 [00:08<00:00,  2.58it/s]


In [60]:
cpa.unstable_host

False

In [9]:
chempots = cpa.chempots

In [12]:
chempots['limits']

{'AlClO-Cl2-Al2O3-CrCl3': {'Al': -9.98493,
  'Cl': -1.79927,
  'O': -5.81225,
  'Cr': -14.3355},
 'AlClO-AlCl3-Cl2-CrCl3': {'Al': -9.91495,
  'Cl': -1.79927,
  'O': -5.88223,
  'Cr': -14.3355},
 'AlClO-Al-Al2O3-Al45Cr7': {'Al': -3.7375,
  'Cl': -3.88175,
  'O': -9.9772,
  'Cr': -10.56994},
 'AlClO-AlCl3-Al-Al45Cr7': {'Al': -3.7375,
  'Cl': -3.85842,
  'O': -10.00053,
  'Cr': -10.56994}}

In [6]:
formation_energy = cpa.get_formation_energy_df(prune_polymorphs=True, include_dft_energies=True)

In [7]:
formation_energy

,Space Group,Energy above Hull (eV/atom),Formation Energy (eV/fu),Formation Energy (eV/atom),DFT Energy (eV/fu),DFT Energy (eV/atom),k-points
Formula,,,,,,,
AlClO,Pmmn,0.000,-7.117,-2.372,-17.596,-5.865,7x6x2
Cr,Im-3m,0.000,0.000,0.000,-9.635,-9.635,12x12x12
Al,Fm-3m,0.000,0.000,0.000,-3.737,-3.737,14x14x14
O2,P4/mmm,0.000,0.000,0.000,-9.886,-4.943,1x1x1
Cl2,P4/mmm,0.000,0.000,0.000,-3.599,-1.799,1x1x1
Al45Cr7,C2/m,0.000,-6.547,-0.126,-242.177,-4.657,3x3x3
CrCl3,R-3,0.000,-4.701,-1.175,-19.733,-4.933,4x4x3
Al2O3,R-3c,0.000,-15.102,-3.020,-37.407,-7.481,5x5x5
AlCl3,C2/m,0.000,-6.177,-1.544,-15.313,-3.828,4x4x4


In [10]:
formation_energies = {}
for key in chempots['limits'].keys():
    def_form_e = -352.20748489 + chempots['limits'][key]['Al'] - (-351.69390352) - chempots['limits'][key]['Cr']
    formation_energies[key] = def_form_e

In [11]:
formation_energies

{'AlClO-Cl2-Al2O3-CrCl3': 3.8369886299999827,
 'AlClO-AlCl3-Cl2-CrCl3': 3.9069686300000264,
 'AlClO-Al-Al2O3-Al45Cr7': 6.318858629999992,
 'AlClO-AlCl3-Al-Al45Cr7': 6.318858629999992}